In [1]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeTwoModelsByScores, MergeThreeModelsByScores

In [2]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [3]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_stacked, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13922 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13913 ( 0.5%) Users that have less than 1 test interactions


In [4]:
stacked_train = combine_matrices(ICM=ICM, URM= URM_train)

In [5]:
### RP3Beta Recommender
RP3Beta_recommender = RP3betaRecommender(URM_train=URM_train)
RP3Beta_recommender.fit(topK=181, alpha=0.5709402717259106, beta=0.3566066452521787, implicit=True, normalize_similarity=True)

RP3betaRecommender: URM Detected 3 ( 0.0%) users with no interactions.


In [30]:
##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=stacked_train)
SLIME_recommender.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)

SLIMElasticNetRecommender: URM Detected 3 ( 0.0%) users with no interactions.


 99%|████████████████████████████████████▊| 17968/18059 [08:22<00:01, 51.11it/s]

In [7]:
##P3ALPHA Recommender
P3Alpha_recommender = P3alphaRecommender(URM_train=stacked_train)
P3Alpha_recommender.fit(topK=46,alpha=0.7723235971713814, implicit=True, normalize_similarity=False)

P3alphaRecommender: URM Detected 3 ( 0.0%) users with no interactions.


100%|████████████████████████████████████▉| 18056/18059 [12:50<00:00, 40.25it/s]

In [8]:
##RP3_P3A
similarity_rp3= RP3Beta_recommender.W_sparse
similarity_p3a=P3Alpha_recommender.W_sparse
RP3_P3A = ItemKNNSimilarityHybridRecommender(URM_train = stacked_train, Similarity_1 = similarity_rp3, Similarity_2=similarity_p3a )
RP3_P3A.fit(alpha=0.9849405466567409)

ItemKNNSimilarityHybridRecommender: URM Detected 3 ( 0.0%) users with no interactions.


In [9]:
similarity_slime = SLIME_recommender.W_sparse
similarity_rp3hp3a = RP3_P3A.W_sparse

In [20]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from skopt.space import Real,Integer
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

hyperparameters_range_dictionary = {
    "alpha": Real(0.6, 0.87),
    "topK":Integer(300,550)
}

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[stacked_train, similarity_slime,similarity_rp3hp3a],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=ItemKNNSimilarityHybridRecommender,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 200
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=ItemKNNSimilarityHybridRecommender.RECOMMENDER_NAME,
                    save_model="best"
                    )





Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 0.6270256865763771, 'topK': 316}
ItemKNNSimilarityHybridRecommender: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 10.86 sec. Users per second: 1281
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 0.6270256865763771, 'topK': 316} - results: PRECISION: 0.2354057, PRECISION_RECALL_MIN_DEN: 0.2405714, RECALL: 0.0690542, MAP: 0.1156918, MAP_MIN_DEN: 0.1179358, MRR: 0.4632777, NDCG: 0.2454196, F1: 0.1067842, HIT_RATE: 0.9046216, ARHR_ALL_HITS: 0.7294768, NOVELTY: 0.0057502, AVERAGE_POPULARITY: 0.4647860, DIVERSITY_MEAN_INTER_LIST: 0.9326052, DIVERSITY_HERFINDAHL: 0.9932538, COVERAGE_ITEM: 0.1890470, COVERAGE_ITEM_CORRECT: 0.1026635, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.9000286, DIVERSITY_GINI: 0.0219890, SHANNON_ENTROPY: 8.6224710, RATIO_DIVERSITY_HERFINDAHL: 0.9936203, RATIO_DIVERSITY_GINI: 0.0823049, R

EvaluatorHoldout: Processed 13913 (100.0%) in 10.76 sec. Users per second: 1294
SearchBayesianSkopt: Config 5 is suboptimal. Config: {'alpha': 0.6991625367126366, 'topK': 526} - results: PRECISION: 0.2368073, PRECISION_RECALL_MIN_DEN: 0.2420226, RECALL: 0.0694238, MAP: 0.1166333, MAP_MIN_DEN: 0.1188516, MRR: 0.4653271, NDCG: 0.2468516, F1: 0.1073702, HIT_RATE: 0.9056278, ARHR_ALL_HITS: 0.7339231, NOVELTY: 0.0057499, AVERAGE_POPULARITY: 0.4613335, DIVERSITY_MEAN_INTER_LIST: 0.9379075, DIVERSITY_HERFINDAHL: 0.9937840, COVERAGE_ITEM: 0.1881057, COVERAGE_ITEM_CORRECT: 0.1026635, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.9010297, DIVERSITY_GINI: 0.0222619, SHANNON_ENTROPY: 8.6799246, RATIO_DIVERSITY_HERFINDAHL: 0.9941507, RATIO_DIVERSITY_GINI: 0.0833264, RATIO_SHANNON_ENTROPY: 0.6954939, RATIO_AVERAGE_POPULARITY: 2.3995745, RATIO_NOVELTY: 0.0303646, 

Iteration No: 6 ended. Evaluation done at random point.
Time taken: 11.7075
Function value obtained: -0.1166
Current minimum: -0.116

EvaluatorHoldout: Processed 13913 (100.0%) in 10.81 sec. Users per second: 1287
SearchBayesianSkopt: Config 10 is suboptimal. Config: {'alpha': 0.7353253698653376, 'topK': 416} - results: PRECISION: 0.2366851, PRECISION_RECALL_MIN_DEN: 0.2418639, RECALL: 0.0693322, MAP: 0.1166423, MAP_MIN_DEN: 0.1188488, MRR: 0.4653897, NDCG: 0.2467788, F1: 0.1072481, HIT_RATE: 0.9054841, ARHR_ALL_HITS: 0.7338963, NOVELTY: 0.0057547, AVERAGE_POPULARITY: 0.4581064, DIVERSITY_MEAN_INTER_LIST: 0.9400995, DIVERSITY_HERFINDAHL: 0.9940032, COVERAGE_ITEM: 0.1885486, COVERAGE_ITEM_CORRECT: 0.1035495, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.9008867, DIVERSITY_GINI: 0.0225776, SHANNON_ENTROPY: 8.7144380, RATIO_DIVERSITY_HERFINDAHL: 0.9943699, RATIO_DIVERSITY_GINI: 0.0845082, RATIO_SHANNON_ENTROPY: 0.6982593, RATIO_AVERAGE_POPULARITY: 2.3827887, RATIO_NOVELTY: 0.0303904, 

Iteration No: 11 ended. Evaluation done at random point.
Time taken: 11.9603
Function value obtained: -0.1166
Current minimum: -0.1

EvaluatorHoldout: Processed 13913 (100.0%) in 10.63 sec. Users per second: 1309
SearchBayesianSkopt: New best config found. Config 16: {'alpha': 0.8023408231177918, 'topK': 533} - results: PRECISION: 0.2367282, PRECISION_RECALL_MIN_DEN: 0.2419178, RECALL: 0.0693371, MAP: 0.1167382, MAP_MIN_DEN: 0.1189561, MRR: 0.4667155, NDCG: 0.2470414, F1: 0.1072584, HIT_RATE: 0.9064903, ARHR_ALL_HITS: 0.7352647, NOVELTY: 0.0057619, AVERAGE_POPULARITY: 0.4529414, DIVERSITY_MEAN_INTER_LIST: 0.9437464, DIVERSITY_HERFINDAHL: 0.9943679, COVERAGE_ITEM: 0.1891578, COVERAGE_ITEM_CORRECT: 0.1041586, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.9018879, DIVERSITY_GINI: 0.0230407, SHANNON_ENTROPY: 8.7693908, RATIO_DIVERSITY_HERFINDAHL: 0.9947347, RATIO_DIVERSITY_GINI: 0.0862415, RATIO_SHANNON_ENTROPY: 0.7026625, RATIO_AVERAGE_POPULARITY: 2.3559238, RATIO_NOVELTY: 0.0304284, 

EvaluatorHoldout: Processed 13922 (100.0%) in 11.15 sec. Users per second: 1248
SearchBayesianSkopt: Config evaluated with evaluat

EvaluatorHoldout: Processed 13913 (100.0%) in 10.77 sec. Users per second: 1292
SearchBayesianSkopt: Config 22 is suboptimal. Config: {'alpha': 0.6519735066551541, 'topK': 428} - results: PRECISION: 0.2365773, PRECISION_RECALL_MIN_DEN: 0.2417922, RECALL: 0.0694146, MAP: 0.1163379, MAP_MIN_DEN: 0.1185732, MRR: 0.4637768, NDCG: 0.2463772, F1: 0.1073356, HIT_RATE: 0.9054841, ARHR_ALL_HITS: 0.7317154, NOVELTY: 0.0057442, AVERAGE_POPULARITY: 0.4656163, DIVERSITY_MEAN_INTER_LIST: 0.9345002, DIVERSITY_HERFINDAHL: 0.9934433, COVERAGE_ITEM: 0.1882164, COVERAGE_ITEM_CORRECT: 0.1025527, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.9008867, DIVERSITY_GINI: 0.0218624, SHANNON_ENTROPY: 8.6305271, RATIO_DIVERSITY_HERFINDAHL: 0.9938098, RATIO_DIVERSITY_GINI: 0.0818311, RATIO_SHANNON_ENTROPY: 0.6915358, RATIO_AVERAGE_POPULARITY: 2.4218510, RATIO_NOVELTY: 0.0303349, 

Iteration No: 23 ended. Evaluation done at random point.
Time taken: 11.7347
Function value obtained: -0.1163
Current minimum: -0.1

...

Iteration No: 168 ended. Search finished for the next optimal point.
Time taken: 14.4314
Function value obtained: -0.1162
Current minimum: -0.1168
Iteration No: 169 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.8418278515134092, 'topK': 550}
ItemKNNSimilarityHybridRecommender: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 12.39 sec. Users per second: 1123
SearchBayesianSkopt: Config 168 is suboptimal. Config: {'alpha': 0.8418278515134092, 'topK': 550} - results: PRECISION: 0.2366707, PRECISION_RECALL_MIN_DEN: 0.2418891, RECALL: 0.0693587, MAP: 0.1167310, MAP_MIN_DEN: 0.1189517, MRR: 0.4664067, NDCG: 0.2469846, F1: 0.1072784, HIT_RATE: 0.9059872, ARHR_ALL_HITS: 0.7350406, NOVELTY: 0.0057657, AVERAGE_POPULARITY: 0.4501456, DIVERSITY_MEAN_INTER_LIST: 0.9456109, DIVERSITY_HERFINDAHL: 0.9945543, COVERAGE_ITEM: 0.1896008, COVERAGE_ITEM_CORRECT: 0.1046570, COVERAGE_USER: 0.9949228, COVERAGE_US

Iteration No: 175 ended. Search finished for the next optimal point.
Time taken: 15.0219
Function value obtained: -0.1167
Current minimum: -0.1168
Iteration No: 176 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.7849854086729257, 'topK': 446}
ItemKNNSimilarityHybridRecommender: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 10.68 sec. Users per second: 1303
SearchBayesianSkopt: Config 175 is suboptimal. Config: {'alpha': 0.7849854086729257, 'topK': 446} - results: PRECISION: 0.2366132, PRECISION_RECALL_MIN_DEN: 0.2418005, RECALL: 0.0693362, MAP: 0.1167222, MAP_MIN_DEN: 0.1189400, MRR: 0.4666104, NDCG: 0.2469675, F1: 0.1072456, HIT_RATE: 0.9059153, ARHR_ALL_HITS: 0.7351590, NOVELTY: 0.0057599, AVERAGE_POPULARITY: 0.4543099, DIVERSITY_MEAN_INTER_LIST: 0.9428444, DIVERSITY_HERFINDAHL: 0.9942777, COVERAGE_ITEM: 0.1888809, COVERAGE_ITEM_CORRECT: 0.1042693, COVERAGE_USER: 0.9949228, COVERAGE_US

Iteration No: 182 ended. Search finished for the next optimal point.
Time taken: 14.3813
Function value obtained: -0.1167
Current minimum: -0.1168
Iteration No: 183 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.8081412219741563, 'topK': 530}
ItemKNNSimilarityHybridRecommender: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 11.19 sec. Users per second: 1243
SearchBayesianSkopt: Config 182 is suboptimal. Config: {'alpha': 0.8081412219741563, 'topK': 530} - results: PRECISION: 0.2366564, PRECISION_RECALL_MIN_DEN: 0.2418490, RECALL: 0.0693235, MAP: 0.1167379, MAP_MIN_DEN: 0.1189577, MRR: 0.4668183, NDCG: 0.2470265, F1: 0.1072348, HIT_RATE: 0.9064903, ARHR_ALL_HITS: 0.7353548, NOVELTY: 0.0057625, AVERAGE_POPULARITY: 0.4525390, DIVERSITY_MEAN_INTER_LIST: 0.9440082, DIVERSITY_HERFINDAHL: 0.9943940, COVERAGE_ITEM: 0.1892131, COVERAGE_ITEM_CORRECT: 0.1043247, COVERAGE_USER: 0.9949228, COVERAGE_US

Iteration No: 189 ended. Search finished for the next optimal point.
Time taken: 14.6055
Function value obtained: -0.1167
Current minimum: -0.1168
Iteration No: 190 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.6670151780589837, 'topK': 406}
ItemKNNSimilarityHybridRecommender: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 10.70 sec. Users per second: 1300
SearchBayesianSkopt: Config 189 is suboptimal. Config: {'alpha': 0.6670151780589837, 'topK': 406} - results: PRECISION: 0.2368648, PRECISION_RECALL_MIN_DEN: 0.2420839, RECALL: 0.0694294, MAP: 0.1164875, MAP_MIN_DEN: 0.1187146, MRR: 0.4639342, NDCG: 0.2466110, F1: 0.1073830, HIT_RATE: 0.9051247, ARHR_ALL_HITS: 0.7323169, NOVELTY: 0.0057464, AVERAGE_POPULARITY: 0.4640852, DIVERSITY_MEAN_INTER_LIST: 0.9355946, DIVERSITY_HERFINDAHL: 0.9935527, COVERAGE_ITEM: 0.1881057, COVERAGE_ITEM_CORRECT: 0.1026081, COVERAGE_USER: 0.9949228, COVERAGE_US

Iteration No: 196 ended. Search finished for the next optimal point.
Time taken: 14.3916
Function value obtained: -0.1167
Current minimum: -0.1168
Iteration No: 197 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.6290307638037476, 'topK': 395}
ItemKNNSimilarityHybridRecommender: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 10.68 sec. Users per second: 1303
SearchBayesianSkopt: Config 196 is suboptimal. Config: {'alpha': 0.6290307638037476, 'topK': 395} - results: PRECISION: 0.2358585, PRECISION_RECALL_MIN_DEN: 0.2410325, RECALL: 0.0692421, MAP: 0.1159069, MAP_MIN_DEN: 0.1181530, MRR: 0.4630029, NDCG: 0.2457429, F1: 0.1070554, HIT_RATE: 0.9048372, ARHR_ALL_HITS: 0.7300060, NOVELTY: 0.0057421, AVERAGE_POPULARITY: 0.4675744, DIVERSITY_MEAN_INTER_LIST: 0.9327063, DIVERSITY_HERFINDAHL: 0.9932639, COVERAGE_ITEM: 0.1886040, COVERAGE_ITEM_CORRECT: 0.1018883, COVERAGE_USER: 0.9949228, COVERAGE_US

In [31]:

similarity_hybrid = ItemKNNSimilarityHybridRecommender(URM_train = stacked_train,
                                                      Similarity_1 = similarity_slime,
                                                      Similarity_2 = similarity_rp3hp3a)
similarity_hybrid.fit(topK=472, alpha=0.7553063200728594 )

ItemKNNSimilarityHybridRecommender: URM Detected 3 ( 0.0%) users with no interactions.


In [32]:
## IALS Recommender
IALS_recommender = IALSRecommender(URM_train=stacked_train)
IALS_recommender.fit(epochs=10, num_factors=42, alpha=0.7617528864750021, reg=8.926401306541349)

IALSRecommenderLinear: URM Detected 3 ( 0.0%) users with no interactions.
IALSRecommenderLinear: Epoch 1 of 10. Elapsed time 6.49 sec


100%|████████████████████████████████████▉| 18056/18059 [08:32<00:00, 51.11it/s]

IALSRecommenderLinear: Epoch 2 of 10. Elapsed time 13.05 sec
IALSRecommenderLinear: Epoch 3 of 10. Elapsed time 19.36 sec
IALSRecommenderLinear: Epoch 4 of 10. Elapsed time 25.67 sec
IALSRecommenderLinear: Epoch 5 of 10. Elapsed time 31.96 sec
IALSRecommenderLinear: Epoch 6 of 10. Elapsed time 38.31 sec
IALSRecommenderLinear: Epoch 7 of 10. Elapsed time 44.64 sec
IALSRecommenderLinear: Epoch 8 of 10. Elapsed time 51.09 sec
IALSRecommenderLinear: Epoch 9 of 10. Elapsed time 57.38 sec
IALSRecommenderLinear: Epoch 10 of 10. Elapsed time 1.06 min
IALSRecommenderLinear: Terminating at epoch 10. Elapsed time 1.06 min


In [33]:
##UserKNNCF
UserKNNCF_recommender = UserKNNCFRecommender(URM_train = stacked_train)
UserKNNCF_recommender.fit(topK=450, shrink=5.0, similarity="tanimoto")

UserKNNCFRecommender: URM Detected 3 ( 0.0%) users with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 14318 (100.0%), 920.25 column/sec. Elapsed time 15.56 sec


In [34]:
##IALS_USERCF
IALS_USERCF_RECS=[IALS_recommender, UserKNNCF_recommender]
IALS_USERCF = MergeTwoModelsByScores(URM_train=stacked_train,recommenders=IALS_USERCF_RECS )
IALS_USERCF.fit(alpha=0.16594460134123257, beta=0.7261566731199409)

MergeTwoModelsByScore: URM Detected 3 ( 0.0%) users with no interactions.


In [35]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from skopt.space import Real
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

hyperparameters_range_dictionary = {
    "alpha": Real(0.1, 1.0),
    "beta": Real(0.1, 1.0)
}
recommenders = [similarity_hybrid, IALS_USERCF]

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[stacked_train, recommenders],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=MergeTwoModelsByScores,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 100
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=MergeTwoModelsByScores.RECOMMENDER_NAME,
                    save_model="best"
                    )




Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 0.5321031933918163, 'beta': 0.7275972215663514}
MergeTwoModelsByScore: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 19.62 sec. Users per second: 709
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 0.5321031933918163, 'beta': 0.7275972215663514} - results: PRECISION: 0.2180263, PRECISION_RECALL_MIN_DEN: 0.2280275, RECALL: 0.0713482, MAP: 0.1051866, MAP_MIN_DEN: 0.1108613, MRR: 0.4441499, NDCG: 0.2315032, F1: 0.1075131, HIT_RATE: 0.8915403, ARHR_ALL_HITS: 0.6801331, NOVELTY: 0.0054960, AVERAGE_POPULARITY: 0.5859939, DIVERSITY_MEAN_INTER_LIST: 0.8467322, DIVERSITY_HERFINDAHL: 0.9846671, COVERAGE_ITEM: 0.1802979, COVERAGE_ITEM_CORRECT: 0.0876571, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.8870137, DIVERSITY_GINI: 0.0103094, SHANNON_ENTROPY: 7.1841413, RATIO_DIVERSITY_HERFINDAHL: 0.9850304, RATIO_DIVERSITY_GI

100%|████████████████████████████████████▉| 18056/18059 [10:14<00:00, 29.37it/s]


EvaluatorHoldout: Processed 13922 (100.0%) in 19.48 sec. Users per second: 715
SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'alpha': 0.5321031933918163, 'beta': 0.7275972215663514} - results:
CUTOFF: 10 - PRECISION: 0.2581526, PRECISION_RECALL_MIN_DEN: 0.2668531, RECALL: 0.0712861, MAP: 0.1320846, MAP_MIN_DEN: 0.1374027, MRR: 0.4948446, NDCG: 0.2719840, F1: 0.1117215, HIT_RATE: 0.9293923, ARHR_ALL_HITS: 0.8029058, NOVELTY: 0.0054962, AVERAGE_POPULARITY: 0.5860506, DIVERSITY_MEAN_INTER_LIST: 0.8466528, DIVERSITY_HERFINDAHL: 0.9846592, COVERAGE_ITEM: 0.1793012, COVERAGE_ITEM_CORRECT: 0.0931392, COVERAGE_USER: 0.9955664, COVERAGE_USER_CORRECT: 0.9252717, DIVERSITY_GINI: 0.0102757, SHANNON_ENTROPY: 7.1834464, RATIO_DIVERSITY_HERFINDAHL: 0.9850225, RATIO_DIVERSITY_GINI: 0.0384621, RATIO_SHANNON_ENTROPY: 0.5755860, RATIO_AVERAGE_POPULARITY: 3.0482763, RATIO_NOVELTY: 0.0290251, 


SearchBayesianSkopt: Saving model in logs/MergeTwoModelsByScore

MergeTwoModelsByScore: Sa

EvaluatorHoldout: Processed 13913 (100.0%) in 19.11 sec. Users per second: 728
SearchBayesianSkopt: New best config found. Config 5: {'alpha': 0.8813771776557564, 'beta': 0.24722362226423594} - results: PRECISION: 0.2251779, PRECISION_RECALL_MIN_DEN: 0.2344123, RECALL: 0.0722482, MAP: 0.1090621, MAP_MIN_DEN: 0.1135096, MRR: 0.4497269, NDCG: 0.2370918, F1: 0.1093966, HIT_RATE: 0.8986559, ARHR_ALL_HITS: 0.6972045, NOVELTY: 0.0055151, AVERAGE_POPULARITY: 0.5756315, DIVERSITY_MEAN_INTER_LIST: 0.8562426, DIVERSITY_HERFINDAHL: 0.9856181, COVERAGE_ITEM: 0.1795227, COVERAGE_ITEM_CORRECT: 0.0890415, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.8940932, DIVERSITY_GINI: 0.0113988, SHANNON_ENTROPY: 7.3509899, RATIO_DIVERSITY_HERFINDAHL: 0.9859817, RATIO_DIVERSITY_GINI: 0.0426657, RATIO_SHANNON_ENTROPY: 0.5890107, RATIO_AVERAGE_POPULARITY: 2.9940823, RATIO_NOVELTY: 0.0291250, 

EvaluatorHoldout: Processed 13922 (100.0%) in 19.13 sec. Users per second: 728
SearchBayesianSkopt: Config evaluated

EvaluatorHoldout: Processed 13913 (100.0%) in 19.23 sec. Users per second: 723
SearchBayesianSkopt: Config 11 is suboptimal. Config: {'alpha': 0.8706253830077418, 'beta': 0.7929239644551305} - results: PRECISION: 0.2195788, PRECISION_RECALL_MIN_DEN: 0.2296298, RECALL: 0.0718525, MAP: 0.1060386, MAP_MIN_DEN: 0.1115877, MRR: 0.4456882, NDCG: 0.2328784, F1: 0.1082745, HIT_RATE: 0.8922590, ARHR_ALL_HITS: 0.6840204, NOVELTY: 0.0054970, AVERAGE_POPULARITY: 0.5849601, DIVERSITY_MEAN_INTER_LIST: 0.8475508, DIVERSITY_HERFINDAHL: 0.9847490, COVERAGE_ITEM: 0.1800764, COVERAGE_ITEM_CORRECT: 0.0890415, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.8877288, DIVERSITY_GINI: 0.0104295, SHANNON_ENTROPY: 7.2031782, RATIO_DIVERSITY_HERFINDAHL: 0.9851123, RATIO_DIVERSITY_GINI: 0.0390377, RATIO_SHANNON_ENTROPY: 0.5771670, RATIO_AVERAGE_POPULARITY: 3.0426043, RATIO_NOVELTY: 0.0290292, 

Iteration No: 12 ended. Evaluation done at random point.
Time taken: 19.3484
Function value obtained: -0.1060
Current

...

Iteration No: 96 ended. Search finished for the next optimal point.
Time taken: 0.7463
Function value obtained: -0.1119
Current minimum: -0.1119
Iteration No: 97 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 1.0, 'beta': 0.1}
SearchBayesianSkopt: Config 96 was already explored at index 32. Config: {'alpha': 1.0, 'beta': 0.1} - results: PRECISION: 0.2295192, PRECISION_RECALL_MIN_DEN: 0.2359518, RECALL: 0.0697293, MAP: 0.1119246, MAP_MIN_DEN: 0.1147934, MRR: 0.4560527, NDCG: 0.2401359, F1: 0.1069627, HIT_RATE: 0.8993747, ARHR_ALL_HITS: 0.7115730, NOVELTY: 0.0055537, AVERAGE_POPULARITY: 0.5564218, DIVERSITY_MEAN_INTER_LIST: 0.8737989, DIVERSITY_HERFINDAHL: 0.9873736, COVERAGE_ITEM: 0.1790243, COVERAGE_ITEM_CORRECT: 0.0876571, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.8948084, DIVERSITY_GINI: 0.0133001, SHANNON_ENTROPY: 7.6265945, RATIO_DIVERSITY_HERFINDAHL: 0.9877379, RATIO_DIVERSITY_GINI: 0.0497823, RATIO_SHANNON_ENTROPY: 0.6110940

/Users/gabriele/opt/anaconda3/envs/jupyter/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 97 ended. Search finished for the next optimal point.
Time taken: 0.7386
Function value obtained: -0.1119
Current minimum: -0.1119
Iteration No: 98 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.4551796858803049, 'beta': 0.45967342245308596}
MergeTwoModelsByScore: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 18.97 sec. Users per second: 733
SearchBayesianSkopt: Config 97 is suboptimal. Config: {'alpha': 0.4551796858803049, 'beta': 0.45967342245308596} - results: PRECISION: 0.2191404, PRECISION_RECALL_MIN_DEN: 0.2291830, RECALL: 0.0717049, MAP: 0.1058056, MAP_MIN_DEN: 0.1114299, MRR: 0.4453887, NDCG: 0.2325391, F1: 0.1080536, HIT_RATE: 0.8920434, ARHR_ALL_HITS: 0.6830550, NOVELTY: 0.0054965, AVERAGE_POPULARITY: 0.5853193, DIVERSITY_MEAN_INTER_LIST: 0.8472254, DIVERSITY_HERFINDAHL: 0.9847165, COVERAGE_ITEM: 0.1806302, COVERAGE_ITEM_CORRECT: 0.0885431, COVERAGE_USER: 0.994922

/Users/gabriele/opt/anaconda3/envs/jupyter/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 100 ended. Search finished for the next optimal point.
Time taken: 0.7595
Function value obtained: -0.1119
Current minimum: -0.1119
SearchBayesianSkopt: Search complete. Best config is 32: {'alpha': 1.0, 'beta': 0.1}



In [37]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from skopt.space import Real
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

hyperparameters_range_dictionary = {
    "alpha": Real(0.4, 0.9),
    "beta": Real(0.1, 0.3)
}
recommenders = [similarity_hybrid, IALS_recommender]

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[stacked_train, recommenders],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=MergeTwoModelsByScores,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 100
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=MergeTwoModelsByScores.RECOMMENDER_NAME,
                    save_model="best"
                    )





Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 0.4271411715939188, 'beta': 0.2691217226714692}
MergeTwoModelsByScore: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 13.70 sec. Users per second: 1016
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 0.4271411715939188, 'beta': 0.2691217226714692} - results: PRECISION: 0.2311507, PRECISION_RECALL_MIN_DEN: 0.2362650, RECALL: 0.0679663, MAP: 0.1126031, MAP_MIN_DEN: 0.1148457, MRR: 0.4559804, NDCG: 0.2406631, F1: 0.1050456, HIT_RATE: 0.9004528, ARHR_ALL_HITS: 0.7140205, NOVELTY: 0.0056343, AVERAGE_POPULARITY: 0.4926400, DIVERSITY_MEAN_INTER_LIST: 0.9319548, DIVERSITY_HERFINDAHL: 0.9931888, COVERAGE_ITEM: 0.1466859, COVERAGE_ITEM_CORRECT: 0.0759732, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.8958810, DIVERSITY_GINI: 0.0156189, SHANNON_ENTROPY: 8.3004826, RATIO_DIVERSITY_HERFINDAHL: 0.9935552, RATIO_DIVERSITY_G

EvaluatorHoldout: Processed 13913 (100.0%) in 12.66 sec. Users per second: 1099
SearchBayesianSkopt: New best config found. Config 5: {'alpha': 0.5983844822882803, 'beta': 0.10436562221647623} - results: PRECISION: 0.2353123, PRECISION_RECALL_MIN_DEN: 0.2404877, RECALL: 0.0691513, MAP: 0.1157342, MAP_MIN_DEN: 0.1179815, MRR: 0.4637658, NDCG: 0.2455249, F1: 0.1068906, HIT_RATE: 0.9064185, ARHR_ALL_HITS: 0.7301622, NOVELTY: 0.0056959, AVERAGE_POPULARITY: 0.4752071, DIVERSITY_MEAN_INTER_LIST: 0.9358828, DIVERSITY_HERFINDAHL: 0.9935816, COVERAGE_ITEM: 0.1733762, COVERAGE_ITEM_CORRECT: 0.0920317, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.9018164, DIVERSITY_GINI: 0.0192002, SHANNON_ENTROPY: 8.5202995, RATIO_DIVERSITY_HERFINDAHL: 0.9939481, RATIO_DIVERSITY_GINI: 0.0718664, RATIO_SHANNON_ENTROPY: 0.6827036, RATIO_AVERAGE_POPULARITY: 2.4717364, RATIO_NOVELTY: 0.0300795, 

EvaluatorHoldout: Processed 13922 (100.0%) in 12.78 sec. Users per second: 1090
SearchBayesianSkopt: Config evaluat

EvaluatorHoldout: Processed 13913 (100.0%) in 12.69 sec. Users per second: 1097
SearchBayesianSkopt: Config 11 is suboptimal. Config: {'alpha': 0.8251379378502783, 'beta': 0.183289989987797} - results: PRECISION: 0.2346367, PRECISION_RECALL_MIN_DEN: 0.2398222, RECALL: 0.0690146, MAP: 0.1152691, MAP_MIN_DEN: 0.1175313, MRR: 0.4630135, NDCG: 0.2448527, F1: 0.1066576, HIT_RATE: 0.9067060, ARHR_ALL_HITS: 0.7280654, NOVELTY: 0.0056851, AVERAGE_POPULARITY: 0.4783759, DIVERSITY_MEAN_INTER_LIST: 0.9351086, DIVERSITY_HERFINDAHL: 0.9935041, COVERAGE_ITEM: 0.1697215, COVERAGE_ITEM_CORRECT: 0.0893737, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.9021024, DIVERSITY_GINI: 0.0185831, SHANNON_ENTROPY: 8.4829812, RATIO_DIVERSITY_HERFINDAHL: 0.9938707, RATIO_DIVERSITY_GINI: 0.0695566, RATIO_SHANNON_ENTROPY: 0.6797134, RATIO_AVERAGE_POPULARITY: 2.4882183, RATIO_NOVELTY: 0.0300228, 

Iteration No: 12 ended. Evaluation done at random point.
Time taken: 12.7996
Function value obtained: -0.1153
Current

EvaluatorHoldout: Processed 13922 (100.0%) in 12.91 sec. Users per second: 1078
SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'alpha': 0.8159442486087216, 'beta': 0.10771627791109449} - results:
CUTOFF: 10 - PRECISION: 0.2794713, PRECISION_RECALL_MIN_DEN: 0.2834802, RECALL: 0.0695907, MAP: 0.1460408, MAP_MIN_DEN: 0.1479988, MRR: 0.5160440, NDCG: 0.2896374, F1: 0.1114336, HIT_RATE: 0.9377963, ARHR_ALL_HITS: 0.8620439, NOVELTY: 0.0057074, AVERAGE_POPULARITY: 0.4718953, DIVERSITY_MEAN_INTER_LIST: 0.9367913, DIVERSITY_HERFINDAHL: 0.9936724, COVERAGE_ITEM: 0.1760895, COVERAGE_ITEM_CORRECT: 0.1007808, COVERAGE_USER: 0.9955664, COVERAGE_USER_CORRECT: 0.9336384, DIVERSITY_GINI: 0.0198400, SHANNON_ENTROPY: 8.5595129, RATIO_DIVERSITY_HERFINDAHL: 0.9940390, RATIO_DIVERSITY_GINI: 0.0742613, RATIO_SHANNON_ENTROPY: 0.6858457, RATIO_AVERAGE_POPULARITY: 2.4545104, RATIO_NOVELTY: 0.0301402, 


SearchBayesianSkopt: Saving model in logs/MergeTwoModelsByScore

MergeTwoModelsByScore: 

EvaluatorHoldout: Processed 13913 (100.0%) in 12.99 sec. Users per second: 1071
SearchBayesianSkopt: Config 24 is suboptimal. Config: {'alpha': 0.7367923607170959, 'beta': 0.24400007149050432} - results: PRECISION: 0.2339539, PRECISION_RECALL_MIN_DEN: 0.2391408, RECALL: 0.0688064, MAP: 0.1147741, MAP_MIN_DEN: 0.1170477, MRR: 0.4614021, NDCG: 0.2440368, F1: 0.1063384, HIT_RATE: 0.9055560, ARHR_ALL_HITS: 0.7252553, NOVELTY: 0.0056651, AVERAGE_POPULARITY: 0.4843110, DIVERSITY_MEAN_INTER_LIST: 0.9335540, DIVERSITY_HERFINDAHL: 0.9933487, COVERAGE_ITEM: 0.1627443, COVERAGE_ITEM_CORRECT: 0.0848884, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.9009582, DIVERSITY_GINI: 0.0174441, SHANNON_ENTROPY: 8.4104175, RATIO_DIVERSITY_HERFINDAHL: 0.9937152, RATIO_DIVERSITY_GINI: 0.0652936, RATIO_SHANNON_ENTROPY: 0.6738991, RATIO_AVERAGE_POPULARITY: 2.5190893, RATIO_NOVELTY: 0.0299170, 

Iteration No: 25 ended. Evaluation done at random point.
Time taken: 13.1193
Function value obtained: -0.1148
Curre

...

Iteration No: 96 ended. Search finished for the next optimal point.
Time taken: 0.8347
Function value obtained: -0.1163
Current minimum: -0.1163
Iteration No: 97 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.9, 'beta': 0.1}
SearchBayesianSkopt: Config 96 was already explored at index 30. Config: {'alpha': 0.9, 'beta': 0.1} - results: PRECISION: 0.2361892, PRECISION_RECALL_MIN_DEN: 0.2414585, RECALL: 0.0693637, MAP: 0.1163153, MAP_MIN_DEN: 0.1185549, MRR: 0.4655146, NDCG: 0.2464469, F1: 0.1072348, HIT_RATE: 0.9069935, ARHR_ALL_HITS: 0.7330851, NOVELTY: 0.0057131, AVERAGE_POPULARITY: 0.4698820, DIVERSITY_MEAN_INTER_LIST: 0.9374407, DIVERSITY_HERFINDAHL: 0.9937373, COVERAGE_ITEM: 0.1779722, COVERAGE_ITEM_CORRECT: 0.0959079, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.9023884, DIVERSITY_GINI: 0.0201958, SHANNON_ENTROPY: 8.5822671, RATIO_DIVERSITY_HERFINDAHL: 0.9941040, RATIO_DIVERSITY_GINI: 0.0755931, RATIO_SHANNON_ENTROPY: 0.6876689

/Users/gabriele/opt/anaconda3/envs/jupyter/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 97 ended. Search finished for the next optimal point.
Time taken: 0.8066
Function value obtained: -0.1163
Current minimum: -0.1163
Iteration No: 98 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.9, 'beta': 0.1}
SearchBayesianSkopt: Config 97 was already explored at index 30. Config: {'alpha': 0.9, 'beta': 0.1} - results: PRECISION: 0.2361892, PRECISION_RECALL_MIN_DEN: 0.2414585, RECALL: 0.0693637, MAP: 0.1163153, MAP_MIN_DEN: 0.1185549, MRR: 0.4655146, NDCG: 0.2464469, F1: 0.1072348, HIT_RATE: 0.9069935, ARHR_ALL_HITS: 0.7330851, NOVELTY: 0.0057131, AVERAGE_POPULARITY: 0.4698820, DIVERSITY_MEAN_INTER_LIST: 0.9374407, DIVERSITY_HERFINDAHL: 0.9937373, COVERAGE_ITEM: 0.1779722, COVERAGE_ITEM_CORRECT: 0.0959079, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.9023884, DIVERSITY_GINI: 0.0201958, SHANNON_ENTROPY: 8.5822671, RATIO_DIVERSITY_HERFINDAHL: 0.9941040, RATIO_DIVERSITY_GINI: 0.0755931, RATIO_SHANNON_ENTROPY: 0.6876689

/Users/gabriele/opt/anaconda3/envs/jupyter/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 98 ended. Search finished for the next optimal point.
Time taken: 0.8007
Function value obtained: -0.1163
Current minimum: -0.1163
Iteration No: 99 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.7491289081455131, 'beta': 0.2600510955766505}
MergeTwoModelsByScore: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 12.69 sec. Users per second: 1096
SearchBayesianSkopt: Config 98 is suboptimal. Config: {'alpha': 0.7491289081455131, 'beta': 0.2600510955766505} - results: PRECISION: 0.2337670, PRECISION_RECALL_MIN_DEN: 0.2389508, RECALL: 0.0687282, MAP: 0.1145979, MAP_MIN_DEN: 0.1168735, MRR: 0.4607049, NDCG: 0.2437900, F1: 0.1062257, HIT_RATE: 0.9047653, ARHR_ALL_HITS: 0.7243464, NOVELTY: 0.0056629, AVERAGE_POPULARITY: 0.4849948, DIVERSITY_MEAN_INTER_LIST: 0.9333782, DIVERSITY_HERFINDAHL: 0.9933311, COVERAGE_ITEM: 0.1615815, COVERAGE_ITEM_CORRECT: 0.0843901, COVERAGE_USER: 0.9949228

In [40]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from skopt.space import Real
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

hyperparameters_range_dictionary = {
    "alpha": Real(0, 1.0),
    "beta": Real(0, 1.0)
}
recommenders = [IALS_USERCF, RP3_P3A]

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[stacked_train, recommenders],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=MergeTwoModelsByScores,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 100
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=MergeTwoModelsByScores.RECOMMENDER_NAME,
                    save_model="best"
                    )





Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 0.6810131948369008, 'beta': 0.95890190092682}
MergeTwoModelsByScore: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 16.29 sec. Users per second: 854
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 0.6810131948369008, 'beta': 0.95890190092682} - results: PRECISION: 0.2143607, PRECISION_RECALL_MIN_DEN: 0.2241511, RECALL: 0.0698154, MAP: 0.1034295, MAP_MIN_DEN: 0.1092209, MRR: 0.4411562, NDCG: 0.2282677, F1: 0.1053268, HIT_RATE: 0.8858621, ARHR_ALL_HITS: 0.6717316, NOVELTY: 0.0055048, AVERAGE_POPULARITY: 0.5890140, DIVERSITY_MEAN_INTER_LIST: 0.8401971, DIVERSITY_HERFINDAHL: 0.9840137, COVERAGE_ITEM: 0.1868874, COVERAGE_ITEM_CORRECT: 0.0856083, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.8813644, DIVERSITY_GINI: 0.0103251, SHANNON_ENTROPY: 7.1231855, RATIO_DIVERSITY_HERFINDAHL: 0.9843767, RATIO_DIVERSITY_GINI: 

EvaluatorHoldout: Processed 13913 (100.0%) in 17.02 sec. Users per second: 818
SearchBayesianSkopt: Config 5 is suboptimal. Config: {'alpha': 0.7206895763502192, 'beta': 0.6592322140284237} - results: PRECISION: 0.2139294, PRECISION_RECALL_MIN_DEN: 0.2237634, RECALL: 0.0698049, MAP: 0.1031610, MAP_MIN_DEN: 0.1089404, MRR: 0.4407013, NDCG: 0.2278696, F1: 0.1052627, HIT_RATE: 0.8849996, ARHR_ALL_HITS: 0.6706465, NOVELTY: 0.0055043, AVERAGE_POPULARITY: 0.5881438, DIVERSITY_MEAN_INTER_LIST: 0.8422631, DIVERSITY_HERFINDAHL: 0.9842203, COVERAGE_ITEM: 0.1888809, COVERAGE_ITEM_CORRECT: 0.0850545, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.8805063, DIVERSITY_GINI: 0.0104539, SHANNON_ENTROPY: 7.1408889, RATIO_DIVERSITY_HERFINDAHL: 0.9845834, RATIO_DIVERSITY_GINI: 0.0391291, RATIO_SHANNON_ENTROPY: 0.5721760, RATIO_AVERAGE_POPULARITY: 3.0591640, RATIO_NOVELTY: 0.0290676, 

Iteration No: 6 ended. Evaluation done at random point.
Time taken: 17.1555
Function value obtained: -0.1032
Current m

EvaluatorHoldout: Processed 13913 (100.0%) in 16.78 sec. Users per second: 829
SearchBayesianSkopt: Config 12 is suboptimal. Config: {'alpha': 0.9035814746617514, 'beta': 0.6793328603325436} - results: PRECISION: 0.2138000, PRECISION_RECALL_MIN_DEN: 0.2235604, RECALL: 0.0696958, MAP: 0.1030775, MAP_MIN_DEN: 0.1088676, MRR: 0.4408983, NDCG: 0.2277730, F1: 0.1051230, HIT_RATE: 0.8847840, ARHR_ALL_HITS: 0.6704903, NOVELTY: 0.0055040, AVERAGE_POPULARITY: 0.5877744, DIVERSITY_MEAN_INTER_LIST: 0.8430453, DIVERSITY_HERFINDAHL: 0.9842985, COVERAGE_ITEM: 0.1889363, COVERAGE_ITEM_CORRECT: 0.0852760, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.8802918, DIVERSITY_GINI: 0.0104799, SHANNON_ENTROPY: 7.1475036, RATIO_DIVERSITY_HERFINDAHL: 0.9846616, RATIO_DIVERSITY_GINI: 0.0392262, RATIO_SHANNON_ENTROPY: 0.5727060, RATIO_AVERAGE_POPULARITY: 3.0572422, RATIO_NOVELTY: 0.0290664, 

Iteration No: 13 ended. Evaluation done at random point.
Time taken: 16.9398
Function value obtained: -0.1031
Current

EvaluatorHoldout: Processed 13913 (100.0%) in 16.73 sec. Users per second: 831
SearchBayesianSkopt: Config 19 is suboptimal. Config: {'alpha': 0.9511008880252859, 'beta': 0.7103918221134871} - results: PRECISION: 0.2137785, PRECISION_RECALL_MIN_DEN: 0.2235221, RECALL: 0.0696685, MAP: 0.1030526, MAP_MIN_DEN: 0.1088394, MRR: 0.4407722, NDCG: 0.2277292, F1: 0.1050893, HIT_RATE: 0.8847121, ARHR_ALL_HITS: 0.6703171, NOVELTY: 0.0055040, AVERAGE_POPULARITY: 0.5877603, DIVERSITY_MEAN_INTER_LIST: 0.8430730, DIVERSITY_HERFINDAHL: 0.9843012, COVERAGE_ITEM: 0.1889363, COVERAGE_ITEM_CORRECT: 0.0852207, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.8802203, DIVERSITY_GINI: 0.0104796, SHANNON_ENTROPY: 7.1476786, RATIO_DIVERSITY_HERFINDAHL: 0.9846644, RATIO_DIVERSITY_GINI: 0.0392251, RATIO_SHANNON_ENTROPY: 0.5727200, RATIO_AVERAGE_POPULARITY: 3.0571688, RATIO_NOVELTY: 0.0290664, 

Iteration No: 20 ended. Evaluation done at random point.
Time taken: 16.8874
Function value obtained: -0.1031
Current

EvaluatorHoldout: Processed 13913 (100.0%) in 17.65 sec. Users per second: 788
SearchBayesianSkopt: Config 26 is suboptimal. Config: {'alpha': 0.7662190098353923, 'beta': 0.9242634788085031} - results: PRECISION: 0.2142025, PRECISION_RECALL_MIN_DEN: 0.2240121, RECALL: 0.0697940, MAP: 0.1033156, MAP_MIN_DEN: 0.1090854, MRR: 0.4410474, NDCG: 0.2281040, F1: 0.1052833, HIT_RATE: 0.8860059, ARHR_ALL_HITS: 0.6712896, NOVELTY: 0.0055046, AVERAGE_POPULARITY: 0.5886804, DIVERSITY_MEAN_INTER_LIST: 0.8409882, DIVERSITY_HERFINDAHL: 0.9840928, COVERAGE_ITEM: 0.1876073, COVERAGE_ITEM_CORRECT: 0.0851653, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.8815074, DIVERSITY_GINI: 0.0103715, SHANNON_ENTROPY: 7.1298635, RATIO_DIVERSITY_HERFINDAHL: 0.9844558, RATIO_DIVERSITY_GINI: 0.0388207, RATIO_SHANNON_ENTROPY: 0.5712926, RATIO_AVERAGE_POPULARITY: 3.0619549, RATIO_NOVELTY: 0.0290692, 

Iteration No: 27 ended. Evaluation done at random point.
Time taken: 17.7972
Function value obtained: -0.1033
Current

...

Iteration No: 68 ended. Search finished for the next optimal point.
Time taken: 19.4852
Function value obtained: -0.1024
Current minimum: -0.1039
Iteration No: 69 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.3337472884156926, 'beta': 0.09272934916009158}
MergeTwoModelsByScore: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 17.85 sec. Users per second: 779
SearchBayesianSkopt: Config 68 is suboptimal. Config: {'alpha': 0.3337472884156926, 'beta': 0.09272934916009158} - results: PRECISION: 0.2133688, PRECISION_RECALL_MIN_DEN: 0.2230872, RECALL: 0.0695541, MAP: 0.1026578, MAP_MIN_DEN: 0.1084104, MRR: 0.4395254, NDCG: 0.2271256, F1: 0.1049096, HIT_RATE: 0.8851434, ARHR_ALL_HITS: 0.6681930, NOVELTY: 0.0055039, AVERAGE_POPULARITY: 0.5864556, DIVERSITY_MEAN_INTER_LIST: 0.8454278, DIVERSITY_HERFINDAHL: 0.9845367, COVERAGE_ITEM: 0.1908744, COVERAGE_ITEM_CORRECT: 0.0855529, COVERAGE_USER: 0.99492

Iteration No: 75 ended. Search finished for the next optimal point.
Time taken: 18.1402
Function value obtained: -0.1032
Current minimum: -0.1039
Iteration No: 76 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.2082305506617676, 'beta': 0.4489562126684077}
MergeTwoModelsByScore: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 17.48 sec. Users per second: 796
SearchBayesianSkopt: Config 75 is suboptimal. Config: {'alpha': 0.2082305506617676, 'beta': 0.4489562126684077} - results: PRECISION: 0.2147344, PRECISION_RECALL_MIN_DEN: 0.2244958, RECALL: 0.0698493, MAP: 0.1036429, MAP_MIN_DEN: 0.1093513, MRR: 0.4413743, NDCG: 0.2285348, F1: 0.1054104, HIT_RATE: 0.8854309, ARHR_ALL_HITS: 0.6724946, NOVELTY: 0.0055060, AVERAGE_POPULARITY: 0.5898028, DIVERSITY_MEAN_INTER_LIST: 0.8380074, DIVERSITY_HERFINDAHL: 0.9837947, COVERAGE_ITEM: 0.1850047, COVERAGE_ITEM_CORRECT: 0.0852207, COVERAGE_USER: 0.9949228

Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 17.8917
Function value obtained: -0.1027
Current minimum: -0.1039
Iteration No: 83 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.7845588425725633, 'beta': 0.2693364137886968}
MergeTwoModelsByScore: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 17.24 sec. Users per second: 807
SearchBayesianSkopt: Config 82 is suboptimal. Config: {'alpha': 0.7845588425725633, 'beta': 0.2693364137886968} - results: PRECISION: 0.2134479, PRECISION_RECALL_MIN_DEN: 0.2231901, RECALL: 0.0696094, MAP: 0.1027097, MAP_MIN_DEN: 0.1084637, MRR: 0.4397437, NDCG: 0.2272299, F1: 0.1049821, HIT_RATE: 0.8850715, ARHR_ALL_HITS: 0.6685354, NOVELTY: 0.0055038, AVERAGE_POPULARITY: 0.5866817, DIVERSITY_MEAN_INTER_LIST: 0.8450627, DIVERSITY_HERFINDAHL: 0.9845002, COVERAGE_ITEM: 0.1910405, COVERAGE_ITEM_CORRECT: 0.0855529, COVERAGE_USER: 0.9949228

Iteration No: 88 ended. Search finished for the next optimal point.
Time taken: 19.2520
Function value obtained: -0.1038
Current minimum: -0.1039
Iteration No: 89 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.7059010441291707, 'beta': 0.559967539504054}
MergeTwoModelsByScore: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 17.60 sec. Users per second: 790
SearchBayesianSkopt: Config 88 is suboptimal. Config: {'alpha': 0.7059010441291707, 'beta': 0.559967539504054} - results: PRECISION: 0.2138791, PRECISION_RECALL_MIN_DEN: 0.2236395, RECALL: 0.0697102, MAP: 0.1031227, MAP_MIN_DEN: 0.1089008, MRR: 0.4409687, NDCG: 0.2278339, F1: 0.1051490, HIT_RATE: 0.8850715, ARHR_ALL_HITS: 0.6706732, NOVELTY: 0.0055041, AVERAGE_POPULARITY: 0.5878646, DIVERSITY_MEAN_INTER_LIST: 0.8428695, DIVERSITY_HERFINDAHL: 0.9842809, COVERAGE_ITEM: 0.1889363, COVERAGE_ITEM_CORRECT: 0.0852207, COVERAGE_USER: 0.9949228, 

Iteration No: 95 ended. Search finished for the next optimal point.
Time taken: 22.2505
Function value obtained: -0.1032
Current minimum: -0.1039
Iteration No: 96 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.18142843853935034, 'beta': 0.8164007364905398}
MergeTwoModelsByScore: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 19.46 sec. Users per second: 715
SearchBayesianSkopt: Config 95 is suboptimal. Config: {'alpha': 0.18142843853935034, 'beta': 0.8164007364905398} - results: PRECISION: 0.2151082, PRECISION_RECALL_MIN_DEN: 0.2248230, RECALL: 0.0697743, MAP: 0.1037836, MAP_MIN_DEN: 0.1093504, MRR: 0.4423055, NDCG: 0.2288279, F1: 0.1053700, HIT_RATE: 0.8860778, ARHR_ALL_HITS: 0.6737098, NOVELTY: 0.0055113, AVERAGE_POPULARITY: 0.5908052, DIVERSITY_MEAN_INTER_LIST: 0.8328139, DIVERSITY_HERFINDAHL: 0.9832754, COVERAGE_ITEM: 0.1810731, COVERAGE_ITEM_CORRECT: 0.0846669, COVERAGE_USER: 0.99492

In [41]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from skopt.space import Real
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

hyperparameters_range_dictionary = {
    "alpha": Real(0, 1.0),
    "beta": Real(0, 1.0)
}
recommenders = [SLIME_recommender, IALS_USERCF]

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[stacked_train, recommenders],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=MergeTwoModelsByScores,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 100
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=MergeTwoModelsByScores.RECOMMENDER_NAME,
                    save_model="best"
                    )






Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 0.8501243552583121, 'beta': 0.2656259377309845}
MergeTwoModelsByScore: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 29.91 sec. Users per second: 465
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 0.8501243552583121, 'beta': 0.2656259377309845} - results: PRECISION: 0.2198519, PRECISION_RECALL_MIN_DEN: 0.2293028, RECALL: 0.0713057, MAP: 0.1054871, MAP_MIN_DEN: 0.1100839, MRR: 0.4419373, NDCG: 0.2316223, F1: 0.1076853, HIT_RATE: 0.8940559, ARHR_ALL_HITS: 0.6799415, NOVELTY: 0.0055045, AVERAGE_POPULARITY: 0.5787227, DIVERSITY_MEAN_INTER_LIST: 0.8549009, DIVERSITY_HERFINDAHL: 0.9854839, COVERAGE_ITEM: 0.1743175, COVERAGE_ITEM_CORRECT: 0.0849438, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.8895166, DIVERSITY_GINI: 0.0108012, SHANNON_ENTROPY: 7.3062429, RATIO_DIVERSITY_HERFINDAHL: 0.9858475, RATIO_DIVERSITY_GI

MergeTwoModelsByScore: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 30.72 sec. Users per second: 453
SearchBayesianSkopt: New best config found. Config 5: {'alpha': 0.6406867076042522, 'beta': 0.10923882403937127} - results: PRECISION: 0.2204269, PRECISION_RECALL_MIN_DEN: 0.2283575, RECALL: 0.0695317, MAP: 0.1055372, MAP_MIN_DEN: 0.1091556, MRR: 0.4414115, NDCG: 0.2310128, F1: 0.1057161, HIT_RATE: 0.8940559, ARHR_ALL_HITS: 0.6801088, NOVELTY: 0.0055187, AVERAGE_POPULARITY: 0.5707173, DIVERSITY_MEAN_INTER_LIST: 0.8631232, DIVERSITY_HERFINDAHL: 0.9863061, COVERAGE_ITEM: 0.1718811, COVERAGE_ITEM_CORRECT: 0.0824520, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.8895166, DIVERSITY_GINI: 0.0114802, SHANNON_ENTROPY: 7.4267591, RATIO_DIVERSITY_HERFINDAHL: 0.9866700, RATIO_DIVERSITY_GINI: 0.0429706, RATIO_SHANNON_ENTROPY: 0.5950818, RATIO_AVERAGE_POPULARITY: 2.9685219, RATIO_NOVELTY: 0.0291439, 

EvaluatorHoldout: Processed 13922 (100.0%)

EvaluatorHoldout: Processed 13913 (100.0%) in 20.85 sec. Users per second: 667
SearchBayesianSkopt: New best config found. Config 11: {'alpha': 0.6063264514526454, 'beta': 0.09036230397419677} - results: PRECISION: 0.2205851, PRECISION_RECALL_MIN_DEN: 0.2281218, RECALL: 0.0690048, MAP: 0.1055775, MAP_MIN_DEN: 0.1089690, MRR: 0.4413837, NDCG: 0.2308936, F1: 0.1051241, HIT_RATE: 0.8944153, ARHR_ALL_HITS: 0.6802909, NOVELTY: 0.0055228, AVERAGE_POPULARITY: 0.5685225, DIVERSITY_MEAN_INTER_LIST: 0.8652535, DIVERSITY_HERFINDAHL: 0.9865191, COVERAGE_ITEM: 0.1712166, COVERAGE_ITEM_CORRECT: 0.0818982, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.8898741, DIVERSITY_GINI: 0.0116491, SHANNON_ENTROPY: 7.4576012, RATIO_DIVERSITY_HERFINDAHL: 0.9868831, RATIO_DIVERSITY_GINI: 0.0436026, RATIO_SHANNON_ENTROPY: 0.5975531, RATIO_AVERAGE_POPULARITY: 2.9571057, RATIO_NOVELTY: 0.0291658, 

EvaluatorHoldout: Processed 13922 (100.0%) in 20.33 sec. Users per second: 685
SearchBayesianSkopt: Config evaluate

EvaluatorHoldout: Processed 13913 (100.0%) in 22.00 sec. Users per second: 633
SearchBayesianSkopt: Config 17 is suboptimal. Config: {'alpha': 0.16957166934603699, 'beta': 0.5509463609671366} - results: PRECISION: 0.2145547, PRECISION_RECALL_MIN_DEN: 0.2243595, RECALL: 0.0700638, MAP: 0.1031622, MAP_MIN_DEN: 0.1088976, MRR: 0.4401294, NDCG: 0.2281183, F1: 0.1056328, HIT_RATE: 0.8872278, ARHR_ALL_HITS: 0.6703934, NOVELTY: 0.0054969, AVERAGE_POPULARITY: 0.5867408, DIVERSITY_MEAN_INTER_LIST: 0.8462931, DIVERSITY_HERFINDAHL: 0.9846232, COVERAGE_ITEM: 0.1814608, COVERAGE_ITEM_CORRECT: 0.0852760, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.8827231, DIVERSITY_GINI: 0.0102402, SHANNON_ENTROPY: 7.1694900, RATIO_DIVERSITY_HERFINDAHL: 0.9849865, RATIO_DIVERSITY_GINI: 0.0383292, RATIO_SHANNON_ENTROPY: 0.5744677, RATIO_AVERAGE_POPULARITY: 3.0518663, RATIO_NOVELTY: 0.0290288, 

Iteration No: 18 ended. Evaluation done at random point.
Time taken: 22.2165
Function value obtained: -0.1032
Curren

EvaluatorHoldout: Processed 13913 (100.0%) in 21.97 sec. Users per second: 633
SearchBayesianSkopt: Config 24 is suboptimal. Config: {'alpha': 0.7869707472439599, 'beta': 0.041857812415213476} - results: PRECISION: 0.2207863, PRECISION_RECALL_MIN_DEN: 0.2265429, RECALL: 0.0667735, MAP: 0.1051858, MAP_MIN_DEN: 0.1078507, MRR: 0.4391914, NDCG: 0.2296551, F1: 0.1025364, HIT_RATE: 0.8932653, ARHR_ALL_HITS: 0.6775649, NOVELTY: 0.0055643, AVERAGE_POPULARITY: 0.5461537, DIVERSITY_MEAN_INTER_LIST: 0.8863720, DIVERSITY_HERFINDAHL: 0.9886308, COVERAGE_ITEM: 0.1724902, COVERAGE_ITEM_CORRECT: 0.0813999, COVERAGE_USER: 0.9949228, COVERAGE_USER_CORRECT: 0.8887300, DIVERSITY_GINI: 0.0135844, SHANNON_ENTROPY: 7.7586170, RATIO_DIVERSITY_HERFINDAHL: 0.9889956, RATIO_DIVERSITY_GINI: 0.0508467, RATIO_SHANNON_ENTROPY: 0.6216725, RATIO_AVERAGE_POPULARITY: 2.8407568, RATIO_NOVELTY: 0.0293845, 

Iteration No: 25 ended. Evaluation done at random point.
Time taken: 22.1661
Function value obtained: -0.1052
Curre

...

Iteration No: 73 ended. Search finished for the next optimal point.
Time taken: 20.1720
Function value obtained: -0.1056
Current minimum: -0.1056
Iteration No: 74 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.8801005159298165, 'beta': 0.6336907543929591}
MergeTwoModelsByScore: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 19.52 sec. Users per second: 713
SearchBayesianSkopt: Config 73 is suboptimal. Config: {'alpha': 0.8801005159298165, 'beta': 0.6336907543929591} - results: PRECISION: 0.2175088, PRECISION_RECALL_MIN_DEN: 0.2273522, RECALL: 0.0710501, MAP: 0.1047523, MAP_MIN_DEN: 0.1100982, MRR: 0.4427398, NDCG: 0.2305977, F1: 0.1071118, HIT_RATE: 0.8894559, ARHR_ALL_HITS: 0.6777320, NOVELTY: 0.0054947, AVERAGE_POPULARITY: 0.5846466, DIVERSITY_MEAN_INTER_LIST: 0.8487389, DIVERSITY_HERFINDAHL: 0.9848678, COVERAGE_ITEM: 0.1761449, COVERAGE_ITEM_CORRECT: 0.0854422, COVERAGE_USER: 0.9949228

Iteration No: 80 ended. Search finished for the next optimal point.
Time taken: 20.1461
Function value obtained: -0.1047
Current minimum: -0.1056
Iteration No: 81 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.9278735832759094, 'beta': 0.13893876490152685}
MergeTwoModelsByScore: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 19.61 sec. Users per second: 710
SearchBayesianSkopt: New best config found. Config 80: {'alpha': 0.9278735832759094, 'beta': 0.13893876490152685} - results: PRECISION: 0.2205994, PRECISION_RECALL_MIN_DEN: 0.2281194, RECALL: 0.0689837, MAP: 0.1055831, MAP_MIN_DEN: 0.1089713, MRR: 0.4413348, NDCG: 0.2308906, F1: 0.1051012, HIT_RATE: 0.8944872, ARHR_ALL_HITS: 0.6802700, NOVELTY: 0.0055227, AVERAGE_POPULARITY: 0.5685737, DIVERSITY_MEAN_INTER_LIST: 0.8652084, DIVERSITY_HERFINDAHL: 0.9865146, COVERAGE_ITEM: 0.1712719, COVERAGE_ITEM_CORRECT: 0.0818982, COVERAGE_USER: 0.9949

Iteration No: 86 ended. Search finished for the next optimal point.
Time taken: 20.1269
Function value obtained: -0.1055
Current minimum: -0.1056
Iteration No: 87 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.6904751631923841, 'beta': 0.9139187466181671}
MergeTwoModelsByScore: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 19.48 sec. Users per second: 714
SearchBayesianSkopt: Config 86 is suboptimal. Config: {'alpha': 0.6904751631923841, 'beta': 0.9139187466181671} - results: PRECISION: 0.2163660, PRECISION_RECALL_MIN_DEN: 0.2262777, RECALL: 0.0708234, MAP: 0.1040939, MAP_MIN_DEN: 0.1096920, MRR: 0.4415774, NDCG: 0.2296970, F1: 0.1067155, HIT_RATE: 0.8886653, ARHR_ALL_HITS: 0.6747738, NOVELTY: 0.0054932, AVERAGE_POPULARITY: 0.5865958, DIVERSITY_MEAN_INTER_LIST: 0.8466984, DIVERSITY_HERFINDAHL: 0.9846638, COVERAGE_ITEM: 0.1775846, COVERAGE_ITEM_CORRECT: 0.0848884, COVERAGE_USER: 0.9949228

Iteration No: 93 ended. Search finished for the next optimal point.
Time taken: 26.1245
Function value obtained: -0.1055
Current minimum: -0.1056
Iteration No: 94 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.9040063251113566, 'beta': 0.11058655355504371}
MergeTwoModelsByScore: URM Detected 3 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13913 (100.0%) in 25.66 sec. Users per second: 542
SearchBayesianSkopt: Config 93 is suboptimal. Config: {'alpha': 0.9040063251113566, 'beta': 0.11058655355504371} - results: PRECISION: 0.2206210, PRECISION_RECALL_MIN_DEN: 0.2275522, RECALL: 0.0683530, MAP: 0.1054475, MAP_MIN_DEN: 0.1085173, MRR: 0.4410915, NDCG: 0.2304683, F1: 0.1043700, HIT_RATE: 0.8940559, ARHR_ALL_HITS: 0.6796772, NOVELTY: 0.0055296, AVERAGE_POPULARITY: 0.5649053, DIVERSITY_MEAN_INTER_LIST: 0.8688151, DIVERSITY_HERFINDAHL: 0.9868753, COVERAGE_ITEM: 0.1712719, COVERAGE_ITEM_CORRECT: 0.0818982, COVERAGE_USER: 0.99492

Iteration No: 100 ended. Search finished for the next optimal point.
Time taken: 22.5691
Function value obtained: -0.1049
Current minimum: -0.1056
SearchBayesianSkopt: Search complete. Best config is 80: {'alpha': 0.9278735832759094, 'beta': 0.13893876490152685}



In [ ]:
##NEXT TRY SLIM with IALS_USERCF